<a href="https://colab.research.google.com/github/ekvirika/WalmartRecruiting/blob/main/notebooks/model_experiment_dlinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install required packages
!pip install wandb torch torchvision pandas numpy matplotlib seaborn scikit-learn mlflow neuralforecast dagshub

# Set up Kaggle API
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 25.2 MB/s eta 0:00:00
  Attempting uninstall: sympy
    Found existing installation: sympy 1.11.1
    Uninstalling sympy-1.11.1:
      Successfully uninstalled sympy-1.11.1


In [3]:
# Upload your kaggle.json to Colab and run:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
# Download the dataset
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip -q walmart-recruiting-store-sales-forecasting.zip

walmart-recruiting-store-sales-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)
replace features.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [5]:
!unzip -q train.csv.zip
!unzip -q stores.csv.zip
!unzip -q test.csv.zip
!unzip -q features.csv.zip

replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
unzip:  cannot find or open stores.csv.zip, stores.csv.zip.zip or stores.csv.zip.ZIP.
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
replace features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [6]:
import mlflow
from dagshub import dagshub_logger
import os
import torch


# Set tracking URI manually
mlflow.set_tracking_uri("https://dagshub.com/ekvirika/WalmartRecruiting.mlflow")

# Use your DagsHub credentials
os.environ["MLFLOW_TRACKING_USERNAME"] = "ekvirika"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "0adb1004ddd4221395353efea2d8ead625e26197"

# Optional: set registry if you're using model registry
mlflow.set_registry_uri("https://dagshub.com/ekvirika/WalmartRecruiting.mlflow")
mlflow.set_experiment("NBeats_Training")

# Detect GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
torch.manual_seed(42)

# W&B setup
wandb_project = 'WalmartRecruiting'
wandb_entity = None  # Replace with your W&B entity if using teams


cpu


In [7]:
!wandb login

wandb: Currently logged in as: ellekvirikashvili (ellekvirikashvili-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [8]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import mlflow
import mlflow.pytorch
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
import torch
import wandb
import joblib
import mlflow
import os
from itertools import product
from neuralforecast.models import PatchTST
from neuralforecast import NeuralForecast
from datetime import datetime

import logging
logging.basicConfig(level=logging.WARNING)
for lib in ["neuralforecast", "pytorch_lightning", "lightning_fabric"]:
    logging.getLogger(lib).setLevel(logging.WARNING)

# --- Data loading ---
STORES_PATH = "stores.csv"
FEATURES_PATH = "features.csv"
TRAIN_PATH = "train.csv"
TEST_PATH = "test.csv"

stores = pd.read_csv(STORES_PATH)
features = pd.read_csv(FEATURES_PATH)
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

class NeuralForecastModels:
    def __init__(self, models, model_names=None, freq='W-FRI', one_model=False):
        self.freq = freq
        self.one_model = one_model
        self.models = models
        self.model_names = model_names if model_names else [f"model_{i}" for i in range(len(models))]
        self.nf = NeuralForecast(models=self.models, freq=self.freq)
        self.fitted_df = None

    def fit(self, df):
        self.fitted_df = df.copy()
        self.nf.fit(df=df)

    def predict(self, h=None):
        if h is None:
            h = self.models[0].h
        return self.nf.predict(h=h)

    def cross_validation(self, df, n_windows=1):
        return self.nf.cross_validation(df=df, n_windows=n_windows)

def preprocess(df):
    df = df.copy()
    df['unique_id'] = df['Store'].astype(str) + "_" + df['Dept'].astype(str)
    df.rename(columns={'Date': 'ds', 'Weekly_Sales': 'y'}, inplace=True)
    df['ds'] = pd.to_datetime(df['ds'])
    return df.sort_values(['unique_id', 'ds']).reset_index(drop=True)

def prepare_data_for_cv(df, n_windows=1, h=53):
    min_length = h * (n_windows + 1) + 10
    series_lengths = df.groupby('unique_id').size()
    valid_series = series_lengths[series_lengths >= min_length].index
    filtered_df = df[df['unique_id'].isin(valid_series)].copy()
    print(f"Original series: {len(series_lengths)}, Valid series for CV: {len(valid_series)}")
    print(f"Original data points: {len(df)}, Filtered data points: {len(filtered_df)}")
    return filtered_df

def compute_wmae(y_true, y_pred, is_holiday=None, holiday_weight=5, non_holiday_weight=1):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    if y_true.shape != y_pred.shape:
        min_len = min(len(y_true), len(y_pred))
        y_true = y_true[:min_len]
        y_pred = y_pred[:min_len]
        if is_holiday is not None:
            is_holiday = np.array(is_holiday)[:min_len]
    if is_holiday is not None:
        weights = np.where(np.array(is_holiday), holiday_weight, non_holiday_weight)
    else:
        weights = np.ones_like(y_true)
    abs_errors = np.abs(y_true - y_pred)
    weighted_errors = weights * abs_errors
    return weighted_errors.sum() / weights.sum()

def create_run_name(params, wmae_score=None):
    """Create descriptive run name for MLflow"""
    key_params = ['input_size', 'patch_len', 'hidden_size', 'n_heads']
    name_parts = []

    for param in key_params:
        if param in params:
            name_parts.append(f"{param[:3]}{params[param]}")

    if wmae_score is not None:
        name_parts.append(f"wmae{wmae_score:.4f}")

    timestamp = datetime.now().strftime("%H%M")
    return f"PatchTST_{'_'.join(name_parts)}_{timestamp}"

def run_patchtst_cv(df, param_grid, fixed_params, experiment_name="PatchTST_Hyperparam_Tuning"):
    mlflow.set_experiment(experiment_name)
    results = []
    keys, values = zip(*param_grid.items())

    total_combinations = len(list(product(*values)))
    print(f"Starting hyperparameter tuning with {total_combinations} combinations...")

    for i, vals in enumerate(product(*values), 1):
        params = dict(zip(keys, vals))
        params.update(fixed_params)
        params.update({'enable_progress_bar': False, 'enable_model_summary': False})

        # Create descriptive run name
        run_name = f"Trial_{i:02d}_inp{params['input_size']}_patch{params['patch_len']}_hid{params.get('hidden_size', 'def')}"

        try:
            with mlflow.start_run(nested=True, run_name=run_name):
                # Log parameters with better organization
                mlflow.log_params({
                    "model_type": "PatchTST",
                    "trial_number": i,
                    **{f"param_{k}": str(v) for k, v in params.items()}
                })

                # Add tags for better organization
                mlflow.set_tag("stage", "hyperparameter_tuning")
                mlflow.set_tag("model_family", "transformer")
                mlflow.set_tag("trial_id", f"trial_{i:02d}")

                model = PatchTST(**params)
                nf_model = NeuralForecastModels(models=[model], model_names=['PatchTST'], freq='W-FRI', one_model=True)
                cv_df = nf_model.cross_validation(df, n_windows=1)

                y_true = cv_df['y'].values
                y_pred = cv_df['PatchTST'].values
                is_holiday = None

                if 'IsHoliday' in df.columns:
                    cv_df_with_holiday = cv_df.merge(df[['unique_id', 'ds', 'IsHoliday']], on=['unique_id', 'ds'], how='left')
                    is_holiday = cv_df_with_holiday['IsHoliday'].fillna(False).values

                score = compute_wmae(y_true, y_pred, is_holiday)

                # Log metrics with more context
                mlflow.log_metric("val_wmae", score)
                mlflow.log_metric("data_points", len(y_true))
                mlflow.log_metric("progress_pct", (i / total_combinations) * 100)

                # Update run name with score
                final_run_name = f"{run_name}_wmae{score:.4f}"
                mlflow.set_tag("final_run_name", final_run_name)

                param_summary = " → ".join(f"{k}={v}" for k, v in {
                    'input_size': params['input_size'],
                    'patch_len': params['patch_len'],
                    'hidden_size': params.get('hidden_size', 'default'),
                    'n_heads': params.get('n_heads', 'default')
                }.items())

                print(f"[{i:2d}/{total_combinations}] {param_summary} → WMAE={score:.4f}")
                results.append({'wmae': score, 'trial': i, **params})

        except Exception as e:
            error_msg = str(e)
            print(f"[{i:2d}/{total_combinations}] ERROR: {error_msg}")

            # Log failed runs too for debugging
            with mlflow.start_run(nested=True, run_name=f"FAILED_{run_name}"):
                mlflow.log_params({f"param_{k}": str(v) for k, v in params.items()})
                mlflow.set_tag("stage", "hyperparameter_tuning")
                mlflow.set_tag("status", "failed")
                mlflow.log_param("error_message", error_msg)
            continue

    if not results:
        raise ValueError("No successful runs completed")

    # Find and log best result
    best_result = min(results, key=lambda r: r['wmae'])
    print(f"\nBest result from trial {best_result['trial']}: WMAE = {best_result['wmae']:.4f}")

    return best_result

# Preprocess and prepare data
df = preprocess(train)
df_cv = prepare_data_for_cv(df, n_windows=1, h=53)

print("Tuning PatchTST hyperparameters...")

# Updated parameter grid with correct PatchTST parameters
param_grid = {
    'input_size': [20, 40],
    'patch_len': [8, 16],
    'hidden_size': [64, 128],  # Changed from d_model to hidden_size
    'n_heads': [4, 8],         # Number of attention heads
    'dropout': [0.0, 0.1],
    'learning_rate': [1e-3, 5e-4]
}

fixed_params = {
    'h': 53,
    'max_steps': 25 * 104,
    'random_seed': 42,
    'batch_size': 64
}

# Run hyperparameter tuning
experiment_name = f"PatchTST_Walmart_Tuning_{datetime.now().strftime('%Y%m%d_%H%M')}"
best_result = run_patchtst_cv(df_cv, param_grid, fixed_params, experiment_name)
print(f"\nBest hyperparameters: {best_result}")

# Train final model with enhanced naming
print("\nTraining final PatchTST model...")
final_run_name = f"PatchTST_Final_Model_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

with mlflow.start_run(run_name=final_run_name):
    # Remove trial-specific keys for final model
    final_params = {k: v for k, v in best_result.items() if k not in ['wmae', 'trial']}

    # Enhanced logging for final model
    mlflow.log_params({
        "model_type": "PatchTST",
        "stage": "final_training",
        **{f"final_{k}": str(v) for k, v in final_params.items()}
    })

    # Add comprehensive tags
    mlflow.set_tag("stage", "final_model")
    mlflow.set_tag("model_family", "transformer")
    mlflow.set_tag("dataset", "walmart_sales")
    mlflow.set_tag("validation_method", "cross_validation")
    mlflow.set_tag("best_trial", str(best_result.get('trial', 'unknown')))

    final_model = PatchTST(**final_params)
    nf_model = NeuralForecastModels(models=[final_model], model_names=['PatchTST'], freq='W-FRI', one_model=True)
    nf_model.fit(df_cv)

    # Cross-validation evaluation
    final_cv_df = nf_model.cross_validation(df_cv, n_windows=1)
    y_true = final_cv_df['y'].values
    y_pred = final_cv_df['PatchTST'].values
    is_holiday = None

    if 'IsHoliday' in df.columns:
        final_cv_df_with_holiday = final_cv_df.merge(df[['unique_id', 'ds', 'IsHoliday']], on=['unique_id', 'ds'], how='left')
        is_holiday = final_cv_df_with_holiday['IsHoliday'].fillna(False).values

    wmae_score = compute_wmae(y_true, y_pred, is_holiday)

    # Enhanced metrics logging
    mlflow.log_metrics({
        "final_wmae": wmae_score,
        "val_wmae_final": wmae_score,
        "horizon": final_params['h'],
        "batch_size": final_params['batch_size'],
        "training_series_count": len(df_cv['unique_id'].unique()),
        "training_data_points": len(df_cv),
        "cv_data_points": len(y_true)
    })

    # Save and log model
    model_file = f"patchtst_final_model_{datetime.now().strftime('%Y%m%d_%H%M%S')}.pkl"
    joblib.dump(nf_model, model_file)
    mlflow.log_artifact(model_file)

    print(f"Final model WMAE: {wmae_score:.4f}")
    print("Model saved and logged to MLflow.")

    # W&B logging with enhanced naming
    wandb_run_name = f"PatchTST_Final_WMAE{wmae_score:.4f}_{datetime.now().strftime('%m%d_%H%M')}"
    wandb.init(
        project="Walmart Recruiting - Store Sales Forecasting",
        name=wandb_run_name,
        tags=["final_model", "patchtst", "transformer", "walmart_sales"]
    )
    wandb.config.update(final_params)
    wandb.log({'val_wmae': wmae_score, 'final_wmae': wmae_score})

    artifact = wandb.Artifact(
        name=f"patchtst_final_model_{datetime.now().strftime('%Y%m%d_%H%M')}",
        type="model",
        description=f"Final PatchTST model with WMAE: {wmae_score:.4f}"
    )
    artifact.add_file(model_file)
    wandb.log_artifact(artifact)
    wandb.finish()

    # Cleanup
    if os.path.exists(model_file):
        os.remove(model_file)

    print("Model saved and logged to W&B.")

# --- INFERENCE AND SUBMISSION GENERATION ---
print("\nGenerating predictions for submission...")

# Preprocess test data to match training format
def preprocess_test_data(test_df, train_df):
    """Preprocess test data to match training format"""
    test_processed = test_df.copy()
    test_processed['unique_id'] = test_processed['Store'].astype(str) + "_" + test_processed['Dept'].astype(str)
    test_processed.rename(columns={'Date': 'ds'}, inplace=True)
    test_processed['ds'] = pd.to_datetime(test_processed['ds'])

    # Add dummy y column for prediction (required by neuralforecast)
    test_processed['y'] = 0

    # Get the last date from training data for each series
    train_last_dates = train_df.groupby('unique_id')['ds'].max().reset_index()
    train_last_dates.columns = ['unique_id', 'last_train_date']

    # Merge to get context
    test_with_context = test_processed.merge(train_last_dates, on='unique_id', how='left')

    return test_processed.sort_values(['unique_id', 'ds']).reset_index(drop=True)

# Prepare full dataset for prediction (train + test structure)
def prepare_full_dataset_for_prediction(train_df, test_df):
    """Combine train and test data for prediction"""
    # Get unique series from test that exist in train
    train_series = set(train_df['unique_id'].unique())
    test_series = set(test_df['unique_id'].unique())
    common_series = train_series.intersection(test_series)

    print(f"Series in train: {len(train_series)}")
    print(f"Series in test: {len(test_series)}")
    print(f"Common series: {len(common_series)}")

    # Filter both datasets to common series
    train_filtered = train_df[train_df['unique_id'].isin(common_series)].copy()
    test_processed_filtered = test_df[test_df['unique_id'].isin(common_series)].copy()

    # Combine for prediction context
    full_dataset = pd.concat([train_filtered, test_processed_filtered], ignore_index=True)
    full_dataset = full_dataset.sort_values(['unique_id', 'ds']).reset_index(drop=True)

    return full_dataset

# Process test data
test_processed = preprocess_test_data(test, df)
full_dataset = prepare_full_dataset_for_prediction(df, test_processed)

print(f"Full dataset shape: {full_dataset.shape}")
print(f"Test data shape: {test_processed.shape}")

# Generate predictions using the trained model
inference_run_name = f"PatchTST_Inference_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

with mlflow.start_run(run_name=inference_run_name):
    mlflow.log_params({
        "stage": "inference",
        "model_type": "PatchTST",
        "test_series_count": len(test_filtered['unique_id'].unique()),
        "test_data_points": len(test_filtered),
        "prediction_horizon": final_params['h']
    })

    mlflow.set_tag("stage", "inference")
    mlflow.set_tag("model_family", "transformer")
    mlflow.set_tag("dataset", "walmart_sales")

    print("Generating predictions...")

    try:
        # Make predictions
        predictions_df = nf_model.predict(h=53)

        # Process predictions to match submission format
        def create_submission_file(predictions_df, test_df, output_file="submission.csv"):
            """Create submission file in the required format"""

            # Merge predictions with test data to get Store, Dept, Date information
            # First, extract unique_id components back to Store and Dept
            predictions_processed = predictions_df.copy()
            predictions_processed[['Store', 'Dept']] = predictions_processed['unique_id'].str.split('_', expand=True)
            predictions_processed['Store'] = predictions_processed['Store'].astype(int)
            predictions_processed['Dept'] = predictions_processed['Dept'].astype(int)
            predictions_processed.rename(columns={'ds': 'Date', 'PatchTST': 'Weekly_Sales'}, inplace=True)

            # Merge with original test data to get the exact rows needed
            test_for_merge = test.copy()
            test_for_merge['Date'] = pd.to_datetime(test_for_merge['Date'])

            submission = test_for_merge.merge(
                predictions_processed[['Store', 'Dept', 'Date', 'Weekly_Sales']],
                on=['Store', 'Dept', 'Date'],
                how='left'
            )

            # Handle any missing predictions (fill with median or last known value)
            if submission['Weekly_Sales'].isna().any():
                print(f"Warning: {submission['Weekly_Sales'].isna().sum()} missing predictions filled with median")
                median_sales = predictions_processed['Weekly_Sales'].median()
                submission['Weekly_Sales'].fillna(median_sales, inplace=True)

            # Ensure no negative predictions
            submission['Weekly_Sales'] = submission['Weekly_Sales'].clip(lower=0)

            # Create final submission with required columns
            final_submission = submission[['Id', 'Weekly_Sales']].copy()
            final_submission.to_csv(output_file, index=False)

            print(f"Submission file saved: {output_file}")
            print(f"Submission shape: {final_submission.shape}")
            print(f"Sample predictions:")
            print(final_submission.head(10))

            # Log statistics
            stats = {
                "prediction_count": len(final_submission),
                "mean_prediction": float(final_submission['Weekly_Sales'].mean()),
                "median_prediction": float(final_submission['Weekly_Sales'].median()),
                "std_prediction": float(final_submission['Weekly_Sales'].std()),
                "min_prediction": float(final_submission['Weekly_Sales'].min()),
                "max_prediction": float(final_submission['Weekly_Sales'].max()),
                "zero_predictions": int((final_submission['Weekly_Sales'] == 0).sum())
            }

            return final_submission, stats

        # Create submission file
        submission_filename = f"patchtst_submission_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        submission_df, prediction_stats = create_submission_file(predictions_df, test, submission_filename)

        # Log prediction statistics
        mlflow.log_metrics(prediction_stats)
        mlflow.log_artifact(submission_filename)

        print("\nPrediction Statistics:")
        for key, value in prediction_stats.items():
            print(f"  {key}: {value}")

        # Log to W&B
        wandb_inference_name = f"PatchTST_Inference_{datetime.now().strftime('%m%d_%H%M')}"
        wandb.init(
            project="Walmart Recruiting - Store Sales Forecasting",
            name=wandb_inference_name,
            tags=["inference", "patchtst", "submission"]
        )
        wandb.log(prediction_stats)

        # Create W&B artifact for submission
        submission_artifact = wandb.Artifact(
            name=f"patchtst_submission_{datetime.now().strftime('%Y%m%d_%H%M')}",
            type="submission",
            description=f"PatchTST submission with {prediction_stats['prediction_count']} predictions"
        )
        submission_artifact.add_file(submission_filename)
        wandb.log_artifact(submission_artifact)
        wandb.finish()

        print(f"\nSubmission file '{submission_filename}' created successfully!")
        print("File logged to both MLflow and W&B.")

        # Cleanup
        if os.path.exists(submission_filename):
            print(f"Submission file ready for upload: {submission_filename}")

    except Exception as e:
        error_msg = f"Error during inference: {str(e)}"
        print(error_msg)
        mlflow.log_param("inference_error", error_msg)
        mlflow.set_tag("status", "failed")
        raise

print("\nComplete pipeline finished! Hyperparameter tuning, training, and inference completed successfully!")

2025/08/03 06:27:41 INFO mlflow.tracking.fluent: Experiment with name 'PatchTST_Walmart_Tuning_20250803_0627' does not exist. Creating a new experiment.


Original series: 3331, Valid series for CV: 2827
Original data points: 421570, Filtered data points: 402458
Tuning PatchTST hyperparameters...
Starting hyperparameter tuning with 64 combinations...
[ 1/64] input_size=20 → patch_len=8 → hidden_size=64 → n_heads=4 → WMAE=2936.6718
[ 2/64] input_size=20 → patch_len=8 → hidden_size=64 → n_heads=4 → WMAE=2946.3911
[ 3/64] input_size=20 → patch_len=8 → hidden_size=64 → n_heads=4 → WMAE=2950.8570
[ 4/64] input_size=20 → patch_len=8 → hidden_size=64 → n_heads=4 → WMAE=2951.7497
[ 5/64] input_size=20 → patch_len=8 → hidden_size=64 → n_heads=8 → WMAE=2928.5935
[ 6/64] input_size=20 → patch_len=8 → hidden_size=64 → n_heads=8 → WMAE=2947.9398
[ 7/64] input_size=20 → patch_len=8 → hidden_size=64 → n_heads=8 → WMAE=2952.4183
